In [1]:
# merge and rename
import shutil
import os
path = "/home/kelly/Pictures/fastrcnn/8800_0302_100/data"
despath = "/home/kelly/Pictures/defect/0307_work/8800_DBC_0928-0305"
for root,dirs,files in os.walk(despath):
    for file in files:
        # shutil.copy(os.path.join(root,file),os.path.join(os.path.join(despath,file)))
        os.rename(os.path.join(root,file),os.path.join(despath,root.split("/")[-1], file[:-4]+'.jpg'))

In [15]:
import xml.etree.cElementTree as ET
import cv2
import os
import numpy as np

crop_path = "/home/kelly/Pictures/fastrcnn/image/4300_crop"
ori_path = "/home/kelly/Pictures/fastrcnn/image/merge_image"
xml_path = "/home/kelly/Pictures/fastrcnn/0223/result/xml"
des_path = "/home/kelly/Pictures/fastrcnn/0223/result/picture"
        

In [ ]:
for root,dirs,files in os.walk(crop_path):
    for file in files:
        if os.path.exists(os.path.join(ori_path,file[:-4]+".jpg")):
            print(os.path.join(root,file))
            try:
                cropImg = cv2.imread(os.path.join(root,file))
                oriImg = cv2.imread(os.path.join(ori_path,file[:-4]+".jpg"))
                result = cv2.matchTemplate(oriImg, cropImg, cv2.TM_CCOEFF_NORMED) 
                x_and_y = np.unravel_index(result.argmax(),result.shape)
                print(x_and_y)

                tree = ET.parse('/home/kelly/Pictures/fastrcnn/0124/8800/Annotations/8800AAHFJABXD1A2100001-01.xml')
                tree_root = tree.getroot()
                change_dict = {"filename":file,"width":oriImg.shape[1],"height":oriImg.shape[0],"xmin":x_and_y[1],"ymin":x_and_y[0],\
                               "xmax":x_and_y[1]+cropImg.shape[1],"ymax":x_and_y[0]+cropImg.shape[0],"name":root.split("/")[-1]}
                for key in change_dict.keys():
                    for e in tree_root.iter(key):
                        e.text = str(change_dict[key])
                tree.write(os.path.join(xml_path,file[:-4]+".xml"))
            except:
                print("continue")

In [5]:
# use xml to catch picture
for root,dirs,files in os.walk(crop_path):
    for file in files:
        if os.path.exists(os.path.join(ori_path,file[:-4]+".jpg")) and\
                          os.path.exists(os.path.join(xml_path,file[:-4]+".xml")):
            if not os.path.exists(os.path.join(des_path,root.split("/")[-1])):
                os.mkdir(os.path.join(des_path,root.split("/")[-1]))
            shutil.move(os.path.join(ori_path,file[:-4]+".jpg"),os.path.join(des_path,root.split("/")[-1],file[:-4]+".jpg"))

# Use A little train size to do faster RCNN

In [13]:
# use picture to catch xml
import shutil 
pic_path = "/home/kelly/Pictures/fastrcnn/8800_0302_100/data"
xml_path = "/home/kelly/Pictures/fastrcnn/8800_0124/result/Annotations"
des_xml_path = "/home/kelly/Pictures/fastrcnn/8800_0302_100/result/Annotations"

for root, dirs,files in os.walk(pic_path):
    for file in files:
        
        shutil.copy(os.path.join(xml_path,file[:-4]+".xml"),os.path.join(des_xml_path,file[:-4]+".xml"))

In [123]:
# select the untrain image 
import shutil
original_pic_path = "/home/kelly/Pictures/fastrcnn/8800_0124/picture"
trained_pic_path = "/home/kelly/Pictures/fastrcnn/8800_0302_100/result2/data"
des_path = "/home/kelly/Pictures/fastrcnn/8800_0302_100/result2/demo"

for root, dirs, files in os.walk(original_pic_path):
    for file in files:
        dir = root.split("/")[-1]
        dir_path = os.path.join(trained_pic_path,dir)
        if not os.path.exists(os.path.join(des_path,dir)):
            os.mkdir(os.path.join(des_path,dir))
        trained_file = os.listdir(dir_path)
        if not os.path.exists(os.path.join(des_path,dir)):
            os.mkdir(os.path.join(des_path,dir))
        if not file in trained_file:
            shutil.copy(os.path.join(root,file),os.path.join(des_path,dir,file))


In [102]:
# find red rec
def size(a, b=0, nargout=1):
    s = np.asarray(a).shape
    if s is ():
        return 1 if b else (1,)*nargout
    # a is not a scalar
    try:
        if b:
            return s[b-1]
        elif nargout > 1:
            return s
    except Exception:
        return 1
    return  1
 
def im2double(im):
    info = np.iinfo(im.dtype)
    return im.astype(np.float) / info.max

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

def double(img):
    return img.astype(np.float)

def getImgInRedRect(imgPath):
        ret = ''
        path_arr = imgPath.split(os.path.sep)
        rawImg = imread(imgPath)
        if size(rawImg,3) < 3:
            return ret
#         output = os.path.abspath(os.path.join(targetPath,'redRec'))
#         for index, path_ in enumerate(path_arr):
#             if path_ not in sourcePath.split(os.path.sep):
#                 output = os.path.join(output, path_)
 
        tmpImg = np.copy(rawImg)
        tmpImg = im2double(tmpImg)
        tmpImg[:,:,0] = tmpImg[:,:,0] > 0.8
        tmpImg[:,:,1] = tmpImg[:,:,1] < 0.2
        tmpImg[:,:,2] = tmpImg[:,:,2] < 0.2
        tmpImg = rgb2gray(tmpImg)
        tmpImg = double(tmpImg > 0.8)
        bw = tmpImg > 0.8
        bw = bw.astype(int)
        cleared = clear_border(bw)
        stats = regionprops(label(cleared))
        flag = False
        #print (imgPath)
        #print(output, output2, output3)
        for currentStats in stats:
            minr, minc, maxr, maxc = currentStats.bbox
            if currentStats.area > 1000 and (maxr-minr) > 25 and (maxc-minc) > 25: # red rectangle
                x = int(max(round(minr + 10), 1))
                y = int(min(round(maxc - 10), size(rawImg,2)))
                crop = rawImg[x:int(min(round(maxr - 10), size(rawImg,1))), int(max(round(minc + 10) , 1)):y,:]
                flag = True
                returnX = x
                returnY = int(max(round(minc + 10) , 1))
                #imsave(output, crop)
        if flag:
            return crop
        else:
            return None
                #print ("red rec:", output)
 


In [108]:
# run the faster rcnn demo
import cv2,os
from scipy.misc import imread, imsave
import numpy as np
from skimage.segmentation import clear_border
from skimage.measure import regionprops, label
import xml.etree.cElementTree as ET


# use demooutput to find xml
def create_xml(ori_path,root,file,xml_path):
    print(file )
    imgPath = os.path.join(root,file)
    ori_path = os.path.join(ori_path,root.split('/')[-1])
    oriImg = cv2.imread(os.path.join(ori_path,file[:-4]+".jpg"))
    xml_path = os.path.join(xml_path,root.split('/')[-1])
    if not os.path.exists(os.path.join(xml_path)):
            os.mkdir(os.path.join(xml_path))

    cropImg = getImgInRedRect(imgPath)
    if not cropImg is None :
        oriImg.astype(np.uint8)
        cropImg = cropImg[:,:,:-1]
        result = cv2.matchTemplate(oriImg, cropImg, cv2.TM_CCOEFF_NORMED) 
        x_and_y = np.unravel_index(result.argmax(),result.shape)
        print(x_and_y)

        tree = ET.parse('/home/kelly/Pictures/fastrcnn/8800_0124/result/Annotations/8800AAHFJABXD1A2100001-01.xml')
        tree_root = tree.getroot()
        change_dict = {"filename":"8800TL76B0TAM1BA100001-01","width":oriImg.shape[1],"height":oriImg.shape[0],"xmin":x_and_y[1],"ymin":x_and_y[0],\
                                       "xmax":x_and_y[1]+cropImg.shape[1],"ymax":x_and_y[0]+cropImg.shape[0],"name":root.split('/')[-1]}

        for key in change_dict.keys():
            for e in tree_root.iter(key):
                e.text = str(change_dict[key])
                tree.write(os.path.join(xml_path,file[:-4]+".xml"))


In [ ]:
# use demooutput to create xml
ori_path = "/home/kelly/Pictures/fastrcnn/8800_0302_100/result2/demo"
demooutput_path = "/home/kelly/Pictures/fastrcnn/8800_0302_100/result3/demooutput"
xml_path = "/home/kelly/Pictures/fastrcnn/8800_0302_100/result3/data"
for root, dirs, files in os.walk(demooutput_path):
    for file in files:
        create_xml(ori_path,root,file,xml_path)


In [ ]:
# create result2 dir
import shutil

# img_path = "/home/kelly/Pictures/fastrcnn/8800_0302_100/demo"
# xml_path = "/home/kelly/Pictures/fastrcnn/8800_0302_100/result3/data"
# output_path = "/home/kelly/Pictures/fastrcnn/8800_0302_100/result3"

# for root, dirs, files in os.walk(xml_path):
#     for file in files:
#         shutil.move(os.path.join(root,file),os.path.join(output_path,"Annotations",file))
#         shutil.copy(os.path.join(img_path, root.split("/")[-1], file[:-4]+".jpg"),\
#                     os.path.join(output_path,"JPEGImages",file[:-4]+".jpg"))
#         shutil.copy(os.path.join(img_path, root.split("/")[-1], file[:-4]+".jpg"),\
#                     os.path.join(xml_path, root.split("/")[-1], file[:-4]+".jpg"))
# merge last dir
last_dir = "/home/kelly/Pictures/fastrcnn/8800_0302_100/result2"
now_dir = "/home/kelly/Pictures/fastrcnn/8800_0302_100/result3"
for root,dirs,files in os.walk(os.path.join(last_dir,"Annotations")):
    for file in files:
        print(file)
        shutil.copy(os.path.join(root,file),os.path.join(now_dir,"Annotations",file))
        root_tmp = os.path.join(*(root.split("/")[:-1]))
        shutil.copy(os.path.join('/'+root_tmp,"JPEGImages",file[:-4]+".jpg"),\
                    os.path.join(now_dir,"JPEGImages",file[:-4]+".jpg"))
        


# use the crop image to find no rec and save with xml
#### xml is include x y xmax ymax

In [ ]:
import os,cv2
import numpy as np
import xml.etree.cElementTree as ET
import shutil


crop_image_path = "/home/kelly/Pictures/defect/0307_work"
no_rec_path = "/home/kelly/Pictures/defect/0307_work/no_redRec"
xml_path = "/home/kelly/Pictures/defect/0307_work/xml"

no_rec_arr = os.listdir(no_rec_path)

#8800TL7BQ2NBC138100001-01

for root,dirs,files in os.walk(crop_image_path):
    for file in files:
        print(file)
        if file in no_rec_arr:
            print("!!")
            # create xml
            oriImg = cv2.imread(os.path.join(no_rec_path,file[:-4]+".jpg"))
            if not os.path.exists(os.path.join("/home/kelly/Pictures/defect/0307_work/data",root.split('/')[-1])):
                os.mkdir(os.path.join("/home/kelly/Pictures/defect/0307_work/data",root.split('/')[-1]))
            shutil.copy(os.path.join(no_rec_path,file[:-4]+".jpg"), os.path.join("/home/kelly/Pictures/defect/0307_work/data",root.split('/')[-1],file[:-4]+".jpg"))
            cropImg = cv2.imread(os.path.join(root,file[:-4]+".jpg"))
            
            result = cv2.matchTemplate(oriImg, cropImg, cv2.TM_CCOEFF_NORMED) 
            x_and_y = np.unravel_index(result.argmax(),result.shape)
            print(x_and_y)
            tree = ET.parse('/home/kelly/Pictures/fastrcnn/8800_0124/result/Annotations/8800AAHFJABXD1A2100001-01.xml')
            tree_root = tree.getroot()
            change_dict = {"filename":"8800TL76B0TAM1BA100001-01","width":oriImg.shape[1],"height":oriImg.shape[0],"xmin":x_and_y[1],"ymin":x_and_y[0],\
                                           "xmax":x_and_y[1]+cropImg.shape[1],"ymax":x_and_y[0]+cropImg.shape[0],"name":root.split('/')[-1]}

            for key in change_dict.keys():
                for e in tree_root.iter(key):
                    e.text = str(change_dict[key])
                    tree.write(os.path.join(xml_path,file[:-4]+".xml"))
        else:
            print("the file doesn't exist")
            

8800TL7BW1NAH109100001-01.jpg
!!
(0, 1009)
8800AAHL5ADXW121100001-01.jpg
!!
(305, 850)
8800TL7AH0XAH12H100001-01.jpg
the file doesn't exist
8800TL7AN3SAS1CH100001-01.jpg
the file doesn't exist
8800TL7AU1PAZ189100001-01.jpg
!!
(247, 599)
8800AAHL3ALYD10A100001-01.jpg
!!
(393, 692)
8800TL7AD05AL13H100001-01.jpg
!!
(219, 619)
8800AAHL5AEXU175100001-04.jpg
!!
(382, 560)
8800TL7BM5DAQ11G100001-01.jpg
!!
(535, 701)
8800TL7A20LAY191100001-01.jpg
the file doesn't exist
8800AAHL4ARXD1F4100001-01.jpg
!!
(325, 699)
8800TL7C224AB14A100001-01.jpg
the file doesn't exist
8800TL7BB3BAU109100001-01.jpg
the file doesn't exist
8800TL7BJ3XAZ137100001-01.jpg
the file doesn't exist
8800TL8140FAL1QA100001-01.jpg
!!
(481, 565)
8800TL7AB1ZAH166100001-01.jpg
!!
(251, 507)
8800TL7BB2PAU1PA100001-01.jpg
the file doesn't exist
8800TL7BU0JBB107100002-01.jpg
!!
(234, 633)
8800TL79W28AR132100001-01.jpg
!!
(345, 557)
8800TL7AG07AY1BJ100001-01.jpg
!!
(514, 478)
8800AAHL3ADYB1QC100001-01.jpg
!!
(543, 665)
8800AAHL4ALXN1

(450, 623)
8800TL7AX17BC1N9100001-01.jpg
the file doesn't exist
8800TL7790VAL105100001-01.jpg
the file doesn't exist
8800TL7780ABC199100001-01.jpg
the file doesn't exist
8800TL7AG2FBD162100001-01.jpg
!!
(253, 663)
8800TL8121XAV11A100001-01.jpg
!!
(318, 744)
8800TL7C23YAG13C100001-01.jpg
the file doesn't exist
8800TL7BR2CAY1B2100001-01.jpg
!!
(314, 681)
8800TL7B716BD162100001-01.jpg
the file doesn't exist
8800AAHL6ASXD121100001-01.jpg
!!
(521, 631)
8800TL81A2JBB16A100001-01.jpg
!!
(453, 603)
8800TL7B20XAD1R1100002-01.jpg
the file doesn't exist
8800TL7BB1KAD1E8100001-01.jpg
!!
(124, 627)
8800TL7AJ0TAM1P7100001-01.jpg
!!
(273, 589)
8800TL7AJ2TAZ13J100001-01.jpg
!!
(405, 651)
8800TL7C11EBC1Q6100001-01.jpg
!!
(642, 644)
8800TL7C20XBD1BK100002-01.jpg
the file doesn't exist
8800TL7AD1MAP1A6100001-01.jpg
!!
(493, 625)
8800TL7CH3TAZ1R2100001-01.jpg
the file doesn't exist
8800TL7AB0ZAV1BC100001-01.jpg
!!
(279, 575)
8800TL7BJ4JAZ1C2100001-01.jpg
!!
(379, 604)
8800TL79905AG10A100001-01.jpg
the fil

(275, 555)
8800TL7B716AE1P4100001-01.jpg
the file doesn't exist
8800TL7B60UAN137100001-01.jpg
the file doesn't exist
8800TL7AJ1CAC12F100002-01.jpg
!!
(411, 722)
8800TL7CP1FBA17A100001-01.jpg
!!
(346, 758)
8800TL7C23PAY1SC100001-01.jpg
the file doesn't exist
8800TL7AF29AH17D100001-01.jpg
the file doesn't exist
8800TL7BM5TAK18C100001-01.jpg
the file doesn't exist
Thumb.jpg
the file doesn't exist
8800TL81B0JAX11K100001-01.jpg
!!
(289, 922)
8800TL7BL04AC178100001-01.jpg
!!
(492, 649)
8800TL79V3EBD127100001-01.jpg
!!
(361, 499)
8800TL7BU0JAJ1N7100001-01.jpg
!!
(658, 537)
8800AAHL3AJXH14A100001-01.jpg
!!
(473, 740)
8800TL7C10XAH1H8100002-01.jpg
!!
(369, 540)
8800TL7B119AA17H100001-01.jpg
the file doesn't exist
8800TL7BJ4AAP196100001-01.jpg
!!
(316, 553)
8800TL7AG26AF1L1100001-01.jpg
!!
(414, 657)
8800AAHL5AFYD157100001-01.jpg
!!
(399, 727)
8800AAHL4BPXT11B100001-01.jpg
!!
(451, 735)
8800TL7CA1QAY138100001-01.jpg
the file doesn't exist
8800TL7BB1KAF15A100001-01.jpg
!!
(82, 607)
8800TL8173GAK1

(556, 589)
8800TL79V3ABC1H6100001-01.jpg
!!
(408, 502)
8800TL79V3EAD1N1100001-01.jpg
!!
(457, 578)
8800TL7BL04AA1D9100001-01.jpg
!!
(386, 650)
8800AAHL3AHXK162100001-01.jpg
!!
(372, 722)
8800TL7C50NAE18G100001-01.jpg
the file doesn't exist
8800TL78Q11AE122100001-01.jpg
the file doesn't exist
8800TL7BH31AH1R3100001-01.jpg
!!
(357, 723)
8800AAHL6ASXC1C1100001-01.jpg
!!
(426, 506)
8800TL7BB3TAU1MA100001-01.jpg
!!
(541, 541)
8800TL7AC0MAC1DH100001-01.jpg
!!
(587, 637)
8800TL7B52HAF155100001-01.jpg
the file doesn't exist
8800TL7AD3XAX1CA100001-01.jpg
the file doesn't exist
8800TL7C20HBA1BC100001-01.jpg
the file doesn't exist
8800TL7BE3RBC1FA100001-01.jpg
!!
(190, 496)
8800TL7AK45BD1L8100001-01.jpg
!!
(237, 665)
8800TL7BP22AT1B2100001-01.jpg
the file doesn't exist
8800TL79T0QBC102100001-01.jpg
!!
(353, 591)
8800TL79V1KAM168100001-01.jpg
the file doesn't exist
8800TL7BV14AN1P9100001-01.jpg
!!
(566, 350)
8800TL7AF3KAS1F1100001-01.jpg
!!
(404, 625)
8800TL7AA0HAF183100001-01.jpg
!!
(421, 581)
88

(337, 412)
8800TL7AV1JBB1C8100001-01.jpg
the file doesn't exist
8800TL79V1NAN113100002-01.jpg
the file doesn't exist
8800TL7AX17BC1RA100002-01.jpg
the file doesn't exist
8800TL7CB3MAT1S1100001-01.jpg
the file doesn't exist
8800TL7B515AQ143100001-01.jpg
the file doesn't exist
8800TL7BW1KAA12D100002-01.jpg
!!
(212, 592)
8800TL7AD2RAG1P7100001-01.jpg
!!
(419, 539)
8800TL7BW1KAZ102100002-01.jpg
!!
(363, 526)
8800TL79V3GAZ113100001-01.jpg
!!
(200, 413)
8800TL7B62CAU1P1100001-01.jpg
the file doesn't exist
8800TL7AH0XAG175100001-01.jpg
!!
(336, 641)
8800TL7AG2HBB1A9100001-01.jpg
!!
(320, 539)
8800TL7B629BD116100001-01.jpg
the file doesn't exist
8800TL7C50NAE10J100001-01.jpg
the file doesn't exist
8800TL7B53QAR1P6100001-01.jpg
!!
(295, 621)
8800TL7BB3BBD1D8100001-01.jpg
the file doesn't exist
8800TL7BP1MBC103100001-01.jpg
the file doesn't exist
8800AAHL4ARXE19B100001-01.jpg
!!
(53, 657)
8800TL7BB3VAH173100001-01.jpg
the file doesn't exist
8800TL79T1RAL11E100002-01.jpg
the file doesn't exist
88

...

(592, 712)
8800TL7B53VBB131100001-01.jpg
the file doesn't exist
8800TL77A0BAG1N1100001-01.jpg
the file doesn't exist
8800TL77U1DAG10L100001-01.jpg
the file doesn't exist
8800TL78J20AQ11K100001-01.jpg
the file doesn't exist
8800TL77A0BAG1E2100001-01.jpg
the file doesn't exist
8800TL7BA2JAW11A100001-01.jpg
the file doesn't exist
8800TL77U1ZAD1DA100001-01.jpg
the file doesn't exist
8800TL76N0KAR1TC100001-01.jpg
the file doesn't exist
8800TL7770BAR1DG100001-01.jpg
the file doesn't exist
8800TL76P2ABD10B100001-01.jpg
the file doesn't exist
8800TL7BJ31AY1A8100001-01.jpg
!!
(59, 92)
8800TL77C0EAP1C1100001-01.jpg
the file doesn't exist
8800TL77N0AAP12G100001-01.jpg
the file doesn't exist
8800TL8173JAJ1A3100001-01.jpg
!!
(537, 669)
8800TL81D3BAS194100001-01.jpg
!!
(359, 550)
8800TL77U22AW1C9100001-01.jpg
the file doesn't exist
8800TL8140LAM169100001-01.jpg
!!
(75, 821)
8800TL7791GAL1DH100002-01.jpg
the file doesn't exist
8800TL8190TAJ1M3100001-01.jpg
!!
(759, 659)
8800TL7BB2SAM1AA100001-01.jpg


(306, 771)
8800TL77W13AM10C100001-01.jpg
the file doesn't exist
8800TL77X04AD19K100001-01.jpg
the file doesn't exist
8800TL76P2CAD144100001-01.jpg
the file doesn't exist
8800TL77N09AS1DH100001-01.jpg
the file doesn't exist
8800TL81B2RAP11A100001-01.jpg
!!
(460, 842)
8800TL7AW33AE1A1100001-01.jpg
the file doesn't exist
8800TL81545AV1J2100001-01.jpg
!!
(375, 579)
8800TL8141NAY1QA100001-01.jpg
!!
(583, 629)
8800TL78L0YBA1CL100001-01.jpg
the file doesn't exist
8800TL7B14MAN1K4100001-01.jpg
the file doesn't exist
8800TL77U1XAB11J100001-01.jpg
the file doesn't exist
8800TL7BA26AN151100001-01.jpg
the file doesn't exist
8800TL77M1PAZ162100001-01.jpg
the file doesn't exist
8800TL76M1NAY12F100001-01.jpg
the file doesn't exist
8800TL77X06BB176100001-01.jpg
the file doesn't exist
8800TL7BC39AE1L2100001-01.jpg
the file doesn't exist
8800TL77U1YAK17E100001-01.jpg
the file doesn't exist
8800TL78L0XAT10K100001-01.jpg
the file doesn't exist
8800TL8131DAF177100001-01.jpg
!!
(572, 775)
8800TL77R1TAQ17910

(515, 669)
8800TL77U1ZAK11H100001-01.jpg
the file doesn't exist
8800TL7CX1CAK166100001-01.jpg
!!
(574, 502)
8800TL77L1FAS145100002-01.jpg
the file doesn't exist
8800TL7B53LAM189100001-01.jpg
the file doesn't exist
8800TL77U1YAK129100001-01.jpg
the file doesn't exist
8800TL77X01AP11L100001-01.jpg
the file doesn't exist
8800TL77W0QAG132100001-01.jpg
the file doesn't exist
8800TL77X04BA14B100001-01.jpg
the file doesn't exist
8800TL7810SAY1A7100001-01.jpg
the file doesn't exist
8800TL78L0LBA1D5100001-01.jpg
the file doesn't exist
8800TL76P2CBA1N4100002-01.jpg
the file doesn't exist
8800TL77L1GAW13B100001-01.jpg
the file doesn't exist
8800TL78D1RAP107100001-01.jpg
the file doesn't exist
8800TL81129AB1D6100001-01.jpg
!!
(458, 706)
8800TL77X0ZAV11D100001-01.jpg
the file doesn't exist
8800TL79W10AG10G100002-01.jpg
the file doesn't exist
8800TL7BB1PAR193100001-01.jpg
the file doesn't exist
8800TL8141HAJ1M3100001-01.jpg
!!
(303, 597)
8800TL76S01AB1N4100001-01.jpg
the file doesn't exist
8800TL812

(527, 690)
8800TL7AP2LBD141100001-01.jpg
the file doesn't exist
8800TL77819AA13D100001-01.jpg
the file doesn't exist
8800TL77U1AAB1C3100001-01.jpg
the file doesn't exist
8800TL76P2BAH1T1100002-01.jpg
the file doesn't exist
8800TL77U23AS116100001-01.jpg
the file doesn't exist
8800TL77318AA1R3100001-01.jpg
the file doesn't exist
8800TL77M1NAR102100001-01.jpg
the file doesn't exist
8800TL7790UAY1C3100001-01.jpg
the file doesn't exist
8800TL7AW1PAG1R1100001-01.jpg
the file doesn't exist
8800TL77T1AAD15B100002-01.jpg
the file doesn't exist
880JTL7C422AH1J3100001-01.jpg
the file doesn't exist
8800TL7B92DAL16A100001-01.jpg
the file doesn't exist
8800TL7AW0QAE1M9100001-01.jpg
the file doesn't exist
8800TL7882DAY14D100001-01.jpg
the file doesn't exist
8800TL77U02BD1DH100001-01.jpg
the file doesn't exist
8800TL77X11AJ115100001-01.jpg
the file doesn't exist
8800TL77U01AJ17L100001-01.jpg
the file doesn't exist
8800TL8154SAV1F2100001-01.jpg
!!
(315, 504)
8800TL76U1EAR1DJ100001-01.jpg
the file doesn

(377, 597)
8800TL76P1KAC1H9100001-01.jpg
the file doesn't exist
8800TL7BE1VAD157100001-01.jpg
the file doesn't exist
8800TL8171RAY1N6100001-01.jpg
!!
(814, 673)
8800TL76P2CAT1L3100002-01.jpg
the file doesn't exist
8800TL78D0GAQ198100001-01.jpg
the file doesn't exist
8800TL78E1EAN162100001-01.jpg
the file doesn't exist
8800TL77L1FAP195100001-01.jpg
the file doesn't exist
8800TL77U1ZAV1D6100001-01.jpg
the file doesn't exist
8800TL7AP2LAR103100001-01.jpg
the file doesn't exist
8800TL7CS0FAD1C3100001-01.jpg
!!
(523, 585)
8800TL78E19AU15G100001-01.jpg
the file doesn't exist
8800TL77T1AAV1AD100001-01.jpg
the file doesn't exist
8800TL7B72AAE132100001-01.jpg
the file doesn't exist
8800TL78D0YAL19B100001-01.jpg
the file doesn't exist
8800TL77U09AT13D100001-01.jpg
the file doesn't exist
8800TL77U05AF1DB100001-01.jpg
the file doesn't exist
8800TL78L24AB1DD100001-01.jpg
the file doesn't exist
8800TL7CL25AU1N2100002-01.jpg
!!
(476, 703)
8800TL77L1FAZ128100003-01.jpg
the file doesn't exist
8800TL7AR